In [1]:
from PIL import Image
import requests
import torch
import re
import json
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
!python run.py "./sample-images"

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████████████| 5/5 [00:02<00:00,  1.81it/s]


In [16]:
' { "Folder 2. Campanula hederacea [G]. i. \"Devon & Cornwall\" [JL]": { "Linnaea borealis L.": { "1 sheet": { "Linnaea borealis [JL]": { "Cites Linn. Sp. Pl. 631; Bauh. Pin. 93": "", "This is a Native of Sweden, Siberia, Switzerland, Russia, Canada, & c. This Specimen was gathered & communicated by Jos: Banks Esqr. who discoverd it in Abundance in the Woods in Newfoundland [JL]"": "", "In ScotlandSee English Botany N.433 [? G]"": "" } } }, "Lonicera periclymenum L.": { "1 folder": { "Lonicera periclymenum [TA]": "" } }, "Viburnum lantana L.": { "1 folder": { "Viburnum lantana [TA]": "" } }, "Viburnum opulus L.": { "1 folder": { "Viburnum opulus [TA]": "" } } }, "Caryophyllaceae": { "Arenaria serpyllifolia L.": { "1 folder": { "Arenaria Serpyllifolia [TA]": "" } }, "Bufonia tenuifolia L., Flora Europaea 1: 133 (1964)": { "1 folder": { "Bufonia Tenuifolia [TA]": "" } }, "Cerastium alpinum L.": { "2 folders": { "Folder 1. Cerastium latifolium [G]": { "i. Cites Rays Syn. 349": "", "ii. \"Cerastium alpinum. This was gathered upon Snowdon at the top of the highest Rock calld Clogwyn y Garnedh. June\" [JL]": "", "iii. \"Top of Snowdon\" [JL]": "", "iv. \"Dr. Solander affirms this to be the true Cerastium alpinum. I had it from Snowdon. He found it at Terra del Fuego & named it C. hirtum with a mark of Dubitation, but now thinks them both one\" [JL]": "" }, "Folder 2. Cerastium arvense [TA]; Cerastium latifolium [G]": { "i. \"Clogwyn d\" [JL]": "", "ii. \"Cerast: alpin: Ben Lomond\" [JL]": "" } } }, "Cerastium arvense L.": { "2 folders": { "Folder 1. Cerastium arvense [TA]": { "i. \"Cerastium arvense. This must be right tho the leaves are hoary\" [JL]": "", "ii. \"Cerastium arvense. Very common in Norfolk. Why does Lin. put the word glabris in his Specific Description in opposition to truth & the Testimony of all Authors\" [B]": "" }, "Folder 2. Cerastium arvense [G]": { "See also under C. arcticum Folder 1.": "" } } }, "Cerastium fontanum Baumg., subsp. triviale (Link) Jalas, Flora Europaea 1: 142 (1964)": { "1 sheet": { "Cerastium vulgaris [TA]; C. vulgatum Huds. and C. viscosum Linn. [G]": { "i. \"Pray inform me": "" } } } } } '

' { "Folder 2. Campanula hederacea [G]. i. "Devon & Cornwall" [JL]": { "Linnaea borealis L.": { "1 sheet": { "Linnaea borealis [JL]": { "Cites Linn. Sp. Pl. 631; Bauh. Pin. 93": "", "This is a Native of Sweden, Siberia, Switzerland, Russia, Canada, & c. This Specimen was gathered & communicated by Jos: Banks Esqr. who discoverd it in Abundance in the Woods in Newfoundland [JL]"": "", "In ScotlandSee English Botany N.433 [? G]"": "" } } }, "Lonicera periclymenum L.": { "1 folder": { "Lonicera periclymenum [TA]": "" } }, "Viburnum lantana L.": { "1 folder": { "Viburnum lantana [TA]": "" } }, "Viburnum opulus L.": { "1 folder": { "Viburnum opulus [TA]": "" } } }, "Caryophyllaceae": { "Arenaria serpyllifolia L.": { "1 folder": { "Arenaria Serpyllifolia [TA]": "" } }, "Bufonia tenuifolia L., Flora Europaea 1: 133 (1964)": { "1 folder": { "Bufonia Tenuifolia [TA]": "" } }, "Cerastium alpinum L.": { "2 folders": { "Folder 1. Cerastium latifolium [G]": { "i. Cites Rays Syn. 349": "", "ii. "Cer

In [2]:
prompt = "Process this image in two steps: Step 1: Turn the text in this picture into markdown. Indicate italics. \
            Indent lines which discuss folder contents as unordered lists Step 2: Convert the markdown text that you created in step 1 into JSON. \
            Use the heading texts as keys, and the folder details as nested numbered lists\
            Your output should consist of the markdown text, then a fenced JSON code block"


In [3]:
%%time

#Load the model in half-precision on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# Image
url_1 = "./sample-images/pg132_a.jpeg"
url_2 = "./sample-images/pg132_b.jpeg"
url_3 = "./sample-images/pag 133.jpeg"
image_1 = Image.open(url_1)
image_2 = Image.open(url_2)
image_3 = Image.open(url_3)
conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {"type": "text", "text": prompt},
        ],
    }
]


# Preprocess the inputs
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
# Excepted output: '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n<|im_start|>assistant\n'

inputs = processor(
    text=[text_prompt, text_prompt, text_prompt], images=[image_1, image_2, image_3], padding=True, return_tensors="pt"
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
output_ids = model.generate(**inputs, max_new_tokens=5000) 
## Increasing the number of new tokens, increases the number of words recognised by the model with trade-off of speed
## 1024 new tokens was capable of reading upto 70% of the input image (pg132_a.jpeg)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]

output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)
print(output_text)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:18<00:00,  3.75s/it]


['### Dicotyledones\n\n#### ACERACEAE\n\n**Acer campestre L.**\n\n- 1 folder. Acer campestre [TA]\n\n**Acer pseudoplatanus L.**\n\n- 2 folders.\n\n  - Folder 1. Acer Pseudo-Platanus [G]. i. "Maple. Bulls: [Bulstrode] Park" [JL]\n  - Folder 2. Acer Pseudo-Platanus [TA].\n\n#### AMARANTHACEAE\n\n**Amaranthus lividus L., Flora Europaea 1: 110 (1964)**\n\n- 1 folder. Amaranthus Blitum [TA].\n\n  - i. Cites Ray\'s Syn. 1957. ii. "Blite Amaranth. Aug! It is often found on Dunghills in the neighbourhood of London. I gather\'d this on a Dunghill at Fulham near London" [JL]. iii. "Amaranthus Blitum. Monoec: 5. and:" [JL]\n\n#### ARALIACEAE\n\n**Hedera helix L.**\n\n- 1 folder. Hedera Helix [TA].\n\n#### ARISTOLOCHIACEAE\n\n**Asarum europaeum L.**\n\n- 1 folder. Asarum europaeum [TA].\n\n  - i. "Asarum europaeum Lin: Asarabacca. Anglis. ex horto" [JL].\n\n#### BALSAMINACEAE\n\n**Impatiens noli-tangere L.**\n\n- 1 folder. Impatiens Noli me tangere [G].\n\n#### BETULACEAE\n\n**Alnus glutinosa (L.)

In [59]:
split_text = output_text[0].split("```")

In [71]:
def clean_string(text):
    #Remove starting json word
    cleaned_text = re.sub(r'^json', '', text)
    #Remove any non-ASCII characters
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', cleaned_text)
    #Make sure any delimeter or text are in double quotation such that string can be turned into a json file
    cleaned_text = re.sub(r'(?<=\w)"\s+(\[[^\]]+\])', r' \1"', cleaned_text)
    #Normalise any spaces or indentation
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

In [74]:
text = clean_string(split_text[1])
text = json.loads(text)

In [75]:
with open("./sample_pg132_a.json", "w") as f:
    json.dump(text, f, indent=4)